In [ ]:
# Copyright 2019 Institut für Nachrichtentechnik, RWTH Aachen University
%matplotlib notebook

from ipywidgets import interact, interactive, fixed, interact_manual

import ipywidgets as widgets
from IPython.display import clear_output, display, HTML, Audio
import matplotlib.pyplot as plt
from scipy import signal # convolution, 

from ient_nb.ient_plots import *
from ient_nb.ient_signals import *

def play(s, fs, autoplay=False):
    display(Audio(s, rate=fs, autoplay=autoplay))

<div>
    <img src="ient_nb/figures/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

# Rayleigh-Verteilung

## Einleitung

Ein Gauß-verteilert Rauschprozess mit Leistungsdichte $N_0$ wird über einen Bandpass $h_\mathrm{BP}(t)$ mit Mittenfrequenz $f_0$ und Bandbreite $f_\Delta$ übertragen.

![Blockdiagramm](figures/rayleigh_block_diagram.png)

Die Tiefpass-Hüllkurve $|n_\mathrm{T}|(t)$ ist in diesem Fall Rayleigh-verteilt.

## Rauschprozess $n_0(t)$

In [ ]:
fs = 22050; # sampling rate
(t, deltat) = np.linspace(-10, 10, 20*fs, retstep=True) # t axis in seconds
Amax = 15 # limit of histogram (the total range would be from -Amax to Amax)
(x, deltax) = np.linspace(-Amax, Amax, 4001, retstep=True) # x axis (corresponds to amplitude)

unitstep = lambda t: np.where(t>=0, 1, 0) # epsilon(t)

Gauß-verteiles Rauschen $n_0(t)$ mit Leistungsdichte $N_0$

In [ ]:
# Calculate PDF
gaussian_pdf = lambda x, ms, sigmas: 1/(sigmas*np.sqrt(2*np.pi)) * np.exp(-(x-ms)**2/(2*sigmas**2))

# Sample n0(t)
N0 = 0.0004
deltaf = 1/fs
sigma_n0_squared = N0/deltaf # Variance of n_0(t)
sigma_n0 = np.sqrt(sigma_n0_squared)
n0 = np.random.normal(0, sigma_n0, len(t))

# Estimate p_n0(x) with histogram
pn0_hist, bins = np.histogram(n0, bins=50, range=(-Amax, Amax), density=True)
x0_hist = (bins[:-1] + bins[1:])/2 # x-axis

# Plot both calculated and measured PDFs
fig, ax = plt.subplots(1, 1); 
ient_stem(ax, x0_hist, pn0_hist, 'rwth', markerfmt=" ", label='Geschätzt');
ax.plot(x, gaussian_pdf(x, 0, sigma_n0), 'k--', label='Berechnet'); 
ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow p_{n_0}(x)$'); ax.legend(); ient_axis(ax);

## Bandpassrauschen $n(t)$

Erstelle einen Butterworth-Bandpass $h_\mathrm{BP}(t)$ 4. Ordnung mit Mittenfrequenz $f_0 = 200\,\mathrm{Hz}$, Bandbreite $f_\Delta=50\,\mathrm{Hz}$

In [ ]:
def butter_bandpass(f0, fdelta, fs, order=5):
    nyq = 0.5 * fs # Nyquist frequency
    normal_min = (f0-fdelta/2) / nyq # normalized 
    normal_max = (f0+fdelta/2) / nyq # normalized 
    b, a = signal.butter(order, [normal_min,normal_max], btype='bandpass', analog=False)
    return b, a

# Filter requirements.
order = 4 # order :)
f0 = 1000  # center frequency
fdelta = 200 # band width

b, a = butter_bandpass(f0, fdelta, fs, order) # generate filter coefficients
w, H = signal.freqz(b, a, worN=len(t)) # compute H(f)=H(z=e^(j 2 pi f)) out of b, a
f = 0.5*fs*w/np.pi

# Plot
fig,ax = plt.subplots(1,1); ax.plot(f, np.abs(H), 'rwth'); 
ax.axvline(f0, color='k', linestyle='--',lw=0.5); # cutoff frequency
ax.set_xlabel(r'$\rightarrow f$ [Hz]'); ax.set_ylabel(r'$\uparrow |H_\mathrm{BP}(f)|$'); 
ax.set_xlim([0,5000]); ax.set_ylim([-.25,1.19]); ient_axis(ax); 

$n(t)$ ist das sog. "Bandpassrauschen", ebenfalls Gauß-verteilt mit Varianz $\sigma_n^2=2N_0 f_\Delta$ (Formel (9.24))

In [ ]:
def butter_filter(s, b, a):
    g = signal.lfilter(b, a, s)
    return g

# Filter n0(t) to yield n(t)
n = butter_filter(n0, b, a)

# Calculate standard deviation
sigma_n_squared = 2*N0*fdelta # Equation (9.24)
sigma_n = np.sqrt(sigma_n_squared)

# Calculate PDF
pn = gaussian_pdf(x, 0, sigma_n)

# Estimate PDF
pn_hist,bins = np.histogram(n, bins=50, range=(-5,5), density=True)
x_hist = (bins[:-1] + bins[1:]) / 2

# Plot
fig,axs = plt.subplots(2,1); 
ax = axs[0]; 
ient_stem(ax, x0_hist, pn0_hist, 'rwth', markerfmt=" ", label='Geschätzt');
ax.plot(x, gaussian_pdf(x, 0, sigma_n0), 'k--', label='Berechnet'); 
ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow p_{n_0}(x)$'); 
ax.legend(); ient_axis(ax);

ax = axs[1]; 
ient_stem(ax, x_hist, pn_hist, 'rwth', markerfmt=" ");
ax.plot(x, pn, 'k--'); 
ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow p_n(x)$'); ient_axis(ax);

play(n0,fs); play(n,fs);

## Tiefpass-Hüllkurve $|n_\mathrm{T}(t)|$

Das (komplexwertige) äquivalente Tiefpasssignal (bezüglich Mittenfrequenz $f_0$) lässt sich über die analytische Komponente ${}^k n_+(t)$ berechnen:
$${}^k n_\mathrm{T}(t) = 2 {}^k n_+(t) \mathrm{e}^{-\mathrm{j}2\pi f_0 t}$$
mit analytischer Komponente $s_+(t)$ eines Signals $s(t)$:  $\mathrm{Re}\{s_+(t)\} = \frac{1}{2} s(t)$ 
und $\mathrm{Im}\{s_+(t)\} = \frac{1}{2} s(t) \ast \frac{1}{\pi t}$ 

In [ ]:
def calculate_eq_lp(s, f0): # s: BP signal, f0: center frequency
    # signal.hilbert is NOT calculating the Hilbert transform, 
    # but the analytic component s_plus(t) using the Hilbert transform
    s_plus = 0.5*signal.hilbert(s) 
    sT = 2*s_plus*np.exp(-2*1J*np.pi*f0*t)
    return sT

# Equivalent lowpass signal of n(t)
nT = calculate_eq_lp(n, f0)

# Plot
fig, ax = plt.subplots(1, 1); 
ax.plot(t*1000, n, 'rwth', label=r'${}^k n(t)$'); 
ax.plot(t*1000, np.abs(nT), 'grun', label=r'${}^k n_\mathrm{T}(t)$'); 
ax.set_xlim([0,100]); ax.set_xlabel(r'$\rightarrow t$ [ms]', bbox=ient_wbbox); 
ax.legend(); ient_axis(ax);

Die Einhüllende $|{}^k n_\mathrm{T}(t)|$ ist Rayleigh-verteilt mit Verteilungsdichtefunktion (Formel (9.27)):
$$p_{|n_\mathrm{T}|}(x) = \epsilon(x) \frac{x}{\sigma_n^2} \mathrm{e}^{- \frac{x^2}{2\sigma_n^2}} $$

In [ ]:
# Calculate PDF
rayleigh_pdf = lambda x, sigma: unitstep(x)*x/sigma**2 * np.exp(-x**2/(2*sigma**2))
pnTabs = rayleigh_pdf(x, sigma_n)

# Estimate PDF of |n_T(t)| with histogram
pnTabs_hist, bins_pnTabs = np.histogram(np.abs(nT), bins=50, range=(0,5), density=True)
xT_hist = (bins_pnTabs[:-1] + bins_pnTabs[1:]) / 2

# Plot
fig,axs = plt.subplots(2,1); 
ax = axs[0]; 
ient_stem(ax, x_hist, pn_hist, 'rwth', markerfmt=" ", label='Geschätzt');
ax.plot(x, pn, 'k--'); 
ax.set_ylabel(r'$\uparrow p_n(x)$'); ax.set_xlabel(r'$\rightarrow x$'); 
ax.set_xlim([-5,5]); ax.set_ylim([0,1.05]); ient_axis(ax);

ax = axs[1]; 
ient_stem(ax, xT_hist, pnTabs_hist, 'rwth', markerfmt=" ", label='Geschätzt');
ax.plot(x, pnTabs, 'k--', label='Berechnet'); 
ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow p_{n_0}(x)$'); 
ax.set_xlim([-5,5]); ax.set_ylim([0,1.75]); ax.legend(); ient_axis(ax);

Plotte ${}^k n(t)$, Einhüllende $|{}^k n_\mathrm{T}(t)|$ sowie Verteilungsdichtefunktionen (Abbildung 9.10)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8,4), gridspec_kw = {'width_ratios':[2, 1]}); 
ax = axs[0];
ax.plot(t*1000, n, 'rwth', label=r'${}^k n(t)$'); 
ax.plot(t*1000, np.abs(nT), 'grun', label=r'$|{}^k n_\mathrm{T}(t)|$'); 
ax.plot(t*1000, -np.abs(nT), 'grun', lw=0.5, linestyle='--'); 
ax.set_xlabel(r'$\rightarrow t$ [ms]', bbox=ient_wbbox); 
ax.set_xlim([0,200]); ax.legend(); ient_axis(ax);

ax = axs[1]; 
ax.plot(pn, x, 'rwth', label=r'$p_n(x)$'); 
ax.plot(pnTabs, x, 'grun', label=r'$p_{|n_\mathrm{T}|}(x)$'); 
ax.set_ylabel(r'$\uparrow x$'); 
ax.set_ylim(axs[0].get_ylim()); ax.legend(); ient_axis(ax);
ax.text(0, sigma_n, r'$\sigma_n$', color='grun', fontsize=12, bbox=ient_wbbox);
ax.plot([0, np.max(pnTabs)], [sigma_n, sigma_n], '--', color='grun', lw=0.5);

Verteilungsfunktion der Rayleigh-Verteilung
$$P_{|n_\mathrm{T}|}(x) = \int\limits_{-\infty}^x  p_{|n_\mathrm{T}|}(\xi) \mathrm{d}\xi = \big[1- \mathrm{e}^{- \frac{x^2}{2\sigma_n^2}}\big] \epsilon(x)$$

In [ ]:
# Calculate PF
rayleigh_pf = lambda x, sigma: unitstep(x)*(1-np.exp(-x**2/(2*sigma**2)))
PnTabs = rayleigh_pf(x, sigma_n)

# Estimate PF
PnTabs_hist = np.cumsum(pnTabs_hist)*np.mean(np.diff(bins_pnTabs))

# Plot
fig,axs = plt.subplots(2, 1); 
ax = axs[0]
ient_stem(ax, xT_hist, PnTabs_hist, 'grun', markerfmt=" ", label='Geschätzt');
ax.plot(x, PnTabs, 'k--', label='Berechnet'); 
ax.set_xlim([-5,5]);
ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow P_{|n_\mathrm{T}|}(x)$'); ax.legend(); ient_axis(ax);

ax = axs[1]    
ient_stem(ax, xT_hist, pnTabs_hist, 'grun', markerfmt=" ", label='Geschätzt');
ax.plot(x, pnTabs, 'k--', label='Berechnet'); 
ax.set_xlabel(r'$\rightarrow x$'); ax.set_ylabel(r'$\uparrow p_{|n_\mathrm{T}|}(x) = \frac{\mathrm{d}}{\mathrm{d} x}P_{|n_\mathrm{T}|}(x)$'); 
ax.set_xlim([-5,5]); ax.set_ylim([0,1.8]); ient_axis(ax);
ax.text(sigma_n, -0.01, r'$\sigma_n$', color='rot', fontsize=12, bbox=ient_wbbox, verticalalignment='top', horizontalalignment='center');
ax.plot([sigma_n, sigma_n], [0, np.max(pnTabs)], '--', color='rot', lw=1, );

Mittelwert $m_{|n_\mathrm{T}|}$ 

In [ ]:
m_nTabs = np.sqrt(np.pi/2)*sigma_n

(m_nTabs,np.mean(np.abs(nT)))

Leistung $L_{|n_\mathrm{T}|}^2$

In [ ]:
L_n = sigma_n_squared # m_n = 0
L_nTabs = 2*L_n

(L_nTabs, np.mean(np.abs(nT)**2))

Streuung $\sigma_{|n_\mathrm{T}|}^2$

In [ ]:
sigma_nTabs_squared = sigma_n_squared * (2-np.pi/2)

(sigma_nTabs_squared, np.var(np.abs(nT)))

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources) (OER). Feel free to use the notebook for your own purposes. The code is licensed under the [MIT license](https://opensource.org/licenses/MIT). 

Please attribute the work as follows: 
*Christian Rohlfing, Übungsbeispiele zur Vorlesung "Informationsübertragung"*, gehalten von Jens-Rainer Ohm, 2019, Institut für Nachrichtentechnik, RWTH Aachen University.